In [68]:
#Backend: take audio from user and then extract features from it using audio processing

In [1]:
!pip install flask
!pip install werkzeug
!pip install librosa
!pip install praat-parselmouth
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install joblib
!apt install ffmpeg
!pip install pydub
!pip install numpy
!pip install soundfile

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [22]:
# --- Core Flask ---
from flask import Flask, request, jsonify

# --- For handling file uploads safely ---
from werkzeug.utils import secure_filename
import os

# --- Audio processing ---
import librosa
import parselmouth
import numpy as np
import soundfile as sf

# --- ML model loading ---
import joblib

# --- Data handling ---
import pandas as pd
import matplotlib.pyplot as plt

In [23]:
# main code for audio processing

In [6]:
import numpy as np
import librosa
import parselmouth

def _safe(arr, fn, default=0.0):
    arr = np.asarray(arr, dtype=float)
    arr = arr[~np.isnan(arr)]
    return float(fn(arr)) if arr.size else float(default)

def extract_pd_base_features(audio_path, sr_target=16000):
    y, sr = librosa.load(audio_path, sr=sr_target, mono=True)
    y, _ = librosa.effects.trim(y)
    if len(y) < 0.3 * sr:
        raise ValueError("Audio too short after trimming; record 3–5s sustained vowel.")

    f0, _, _ = librosa.pyin(y, fmin=50, fmax=400, sr=sr)
    f0v = f0[~np.isnan(f0)] if f0 is not None else np.array([])
    mdvp_fo  = _safe(f0v, np.mean)
    mdvp_fhi = _safe(f0v, np.max)
    mdvp_flo = _safe(f0v, np.min)

    snd = parselmouth.Sound(y, sr)
    pp  = parselmouth.praat.call(snd, "To PointProcess (periodic, cc)", 75, 500)

    jitt_pct = parselmouth.praat.call(pp, "Get jitter (local)", 0, 0, 75, 500, 1.3)
    jitt_abs = parselmouth.praat.call(pp, "Get jitter (local, absolute)", 0, 0, 75, 500, 1.3)
    jitt_rap = parselmouth.praat.call(pp, "Get jitter (rap)", 0, 0, 75, 500, 1.3)
    jitt_ppq = parselmouth.praat.call(pp, "Get jitter (ppq5)", 0, 0, 75, 500, 1.3)
    jitt_ddp = 3.0 * jitt_rap

    shim_loc    = parselmouth.praat.call([snd, pp], "Get shimmer (local)", 0, 0, 75, 500, 1.3, 1.6)
    shim_loc_db = parselmouth.praat.call([snd, pp], "Get shimmer (local_dB)", 0, 0, 75, 500, 1.3, 1.6)
    shim_apq3   = parselmouth.praat.call([snd, pp], "Get shimmer (apq3)", 0, 0, 75, 500, 1.3, 1.6)
    shim_apq5   = parselmouth.praat.call([snd, pp], "Get shimmer (apq5)", 0, 0, 75, 500, 1.3, 1.6)
    shim_apq11  = parselmouth.praat.call([snd, pp], "Get shimmer (apq11)", 0, 0, 75, 500, 1.3, 1.6)
    shim_dda    = 3.0 * shim_apq3

    harm = parselmouth.praat.call(snd, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    hnr = parselmouth.praat.call(harm, "Get mean", 0, 0)
    nhr = float(10 ** (-hnr/10.0)) if np.isfinite(hnr) else 0.0

    return {
        "MDVP:Fo(Hz)": mdvp_fo,
        "MDVP:Fhi(Hz)": mdvp_fhi,
        "MDVP:Flo(Hz)": mdvp_flo,
        "MDVP:Jitter(%)": jitt_pct,
        "MDVP:Jitter(Abs)": jitt_abs,
        "MDVP:RAP": jitt_rap,
        "MDVP:PPQ": jitt_ppq,
        "Jitter:DDP": jitt_ddp,
        "MDVP:Shimmer": shim_loc,
        "MDVP:Shimmer(dB)": shim_loc_db,
        "Shimmer:APQ3": shim_apq3,
        "Shimmer:APQ5": shim_apq5,
        "MDVP:APQ": shim_apq11,
        "Shimmer:DDA": shim_dda,
        "NHR": nhr,
        "HNR": hnr
    }


In [8]:
#Load the trained model
import joblib
import pandas as pd
import numpy as np

# load your trained pipeline and feature order
bundle = joblib.load("model.pkl")   # adjust path if needed
pipe = bundle["pipeline"]
feature_order = bundle["feature_cols"]

In [11]:
#setting up the server
# --- block: imports + config ---
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename
from pydub import AudioSegment
import os, uuid

ALLOWED_EXTS = {"webm", "mp4", "m4a", "mp3", "ogg", "wav"}
UPLOAD_DIR   = os.path.join(os.getcwd(), "uploads")
MAX_MB       = 16

os.makedirs(UPLOAD_DIR, exist_ok=True)

app = Flask(__name__)
app.config["MAX_CONTENT_LENGTH"] = MAX_MB * 1024 * 1024  # 16 MB


In [12]:
# --- block: helpers ---
def _unique_name(prefix: str, ext: str) -> str:
    return f"{prefix}_{uuid.uuid4().hex}.{ext}"

def _allowed(filename: str) -> bool:
    return "." in filename and filename.rsplit(".", 1)[1].lower() in ALLOWED_EXTS

def convert_to_wav(src_path: str, dst_dir: str) -> str:
    """
    Convert any supported input (webm/mp4/m4a/mp3/ogg/wav) to 16 kHz, mono, 16-bit PCM WAV.
    Returns absolute destination path.
    """
    dst_path = os.path.join(dst_dir, _unique_name("voice", "wav"))
    audio = AudioSegment.from_file(src_path)
    audio = audio.set_channels(1).set_frame_rate(16000).set_sample_width(2)
    audio.export(dst_path, format="wav")
    return dst_path


In [13]:
# --- block: health ---
@app.route("/", methods=["GET"])
def home():
    return "Send POST /upload with form-data key 'file' (webm/mp4/m4a/mp3/ogg/wav)."


In [14]:
# --- block: upload ---
@app.route("/upload", methods=["POST"])
def upload():
    if "file" not in request.files:
        return jsonify({"error": "No file part. Use form-data key 'file'."}), 400

    f = request.files["file"]
    if f.filename == "":
        return jsonify({"error": "Empty filename."}), 400
    if not _allowed(f.filename):
        return jsonify({"error": "Allowed: webm, mp4, m4a, mp3, ogg, wav"}), 400

    # save original upload
    safe = secure_filename(f.filename)
    orig_ext = safe.rsplit(".", 1)[1].lower()
    tmp_name = _unique_name("upload", orig_ext)
    tmp_path = os.path.join(UPLOAD_DIR, tmp_name)
    f.save(tmp_path)

    try:
        # normalize everything to a clean WAV
        wav_path = convert_to_wav(tmp_path, UPLOAD_DIR)

        # optional: delete the original
        try:
            os.remove(tmp_path)
        except Exception:
            pass

        # optional: quick duration metadata
        dur_sec = None
        try:
            from pydub.utils import mediainfo
            dur_sec = float(mediainfo(wav_path).get("duration", 0))
        except Exception:
            pass

        return jsonify({
            "message": "Upload OK",
            "wav_path": wav_path,           # pass this to your feature extractor
            "original_filename": safe,
            "duration_sec": dur_sec
        }), 200

    except Exception as e:
        # cleanup temp if conversion fails
        try:
            if os.path.exists(tmp_path):
                os.remove(tmp_path)
        except Exception:
            pass
        return jsonify({"error": str(e)}), 500


In [15]:
# --- block: vector builder ---
def build_vector_from_wav(wav_path: str) -> pd.DataFrame:
    base = extract_pd_base_features(wav_path)          # dict of core PD features
    full = engineer_like_training(base)                # add engineered fields used in training
    row  = [full.get(col, np.nan) for col in FEATURE_ORDER]
    X    = pd.DataFrame([row], columns=FEATURE_ORDER)
    return X.fillna(0)                                 # simple NaN handling


In [16]:
# --- block: predict ---
@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json(silent=True) or {}
    wav_path = data.get("wav_path")
    if not wav_path or not os.path.exists(wav_path):
        return jsonify({"error": "Valid 'wav_path' required."}), 400

    try:
        X = build_vector_from_wav(wav_path)
        proba = float(PIPE.predict_proba(X)[0][1])
        pred  = int(proba >= 0.5)
        return jsonify({
            "wav_path": wav_path,
            "prediction": pred,
            "label": "Parkinsons" if pred else "Healthy",
            "score": round(proba, 4)
        }), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500


In [ ]:
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)
